# extração do texto do PDF

In [ ]:
import PyPDF2
import nltk

: 

In [5]:
#Listagem de palavras frequentes que não alteram o significado do texto (em ingles)
nltk.download('stopwords')
from nltk.corpus import stopwords
irrelevantes = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#tratamento de flexão de palavras por lematização
tk = nltk.tokenize.WhitespaceTokenizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
lmt = nltk.stem.WordNetLemmatizer()

#A biblioteca nltk não tem dados para lematização em PT
def lematize_text(text):
    lem_words = list()
    for word in tk.tokenize(text):
        lem_words.append(lmt.lemmatize(word))
    return lem_words


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
reader = PyPDF2.PdfReader('ArquivosEN/1608.06902.pdf')

In [8]:
len(reader.pages)

11

In [9]:
textin = reader.pages[0].extract_text()

In [10]:
lematizedTXT = lematize_text(textin)
print(len(lematizedTXT))

541


In [11]:
#Removendo elementos ruins - função bem custosa, deve ter alguma que faz isso mais facil
for word in lematizedTXT:
    for stopword in irrelevantes:
        if word == stopword:
            lematizedTXT.remove(word)


In [12]:
#Deu certo, mas ainda tem palavras que precisariam ser tratadas
print(len(lematizedTXT))
print(lematizedTXT)

413
['Recurrent', 'Neural', 'Networks', 'With', 'Limited', 'Numerical', 'Precision', 'Joachim', 'Ott\x03,', 'Zhouhan', 'Linz,Ying', 'Zhangz,', 'Shih-Chii', 'Liu\x03,', 'Yoshua', 'Bengiozy', '\x03Institute', 'Neuroinformatics,', 'University', 'Zurich', 'ETH', 'Zurich', 'ottj@ethz.ch,', 'shih@ini.ethz.ch', 'zDépartement', 'd’informatique', 'et', 'de', 'recherche', 'opérationnelle,', 'Université', 'de', 'Montréal', 'yCIFAR', 'Senior', 'Fellow', '{zhouhan.lin,', 'ying.zhang}@umontreal.ca', 'Abstract', 'Recurrent', 'Neural', 'Networks', '(RNNs)', 'produce', 'state-of-art', 'performance', 'many', 'machine', 'learning', 'task', 'their', 'demand', 'resource', 'term', 'memory', 'computational', 'power', 'often', 'high.', 'Therefore,', 'great', 'interest', 'optimizing', 'computation', 'performed', 'model', 'especially', 'considering', 'development', 'specialized', 'low-power', 'hardware', 'deep', 'networks.', 'One', 'way', 'reducing', 'computational', 'need', 'limit', 'numerical', 'precision', '

In [ ]:
#Existem caracteres estranhos (Ott\x03,)
#Verificar pontuação, ela pode não ser tão importante
#Existem alguns trechos em espanhol(?)
#Palavras compostas como low-precision
#1arXiv:1608.06902v2 (?)